# Apply Phi3 model with HuggingFace Causal ML

In [3]:
chats = [
    (1, "fix grammar: helol mi friend"),
    (2, "What is SynapseML"),
    (3, "translate to Spanish: hello")
]

chat_df = spark.createDataFrame(chats, ["row_index", "content"])
chat_df.show()

StatementMeta(, f9f20cd1-6e11-4d5d-af98-de2812967df7, 9, Finished, Available, Finished)

+---------+--------------------+
|row_index|             content|
+---------+--------------------+
|        1|fix grammar: helo...|
|        2|   What is SynapseML|
|        3|translate to Span...|
+---------+--------------------+



In [10]:
from synapse.ml.llm.HuggingFaceCausallmTransform import HuggingFaceCausalLM

phi3_transformer = (
    HuggingFaceCausalLM()
    .setModelName("microsoft/Phi-3-mini-4k-instruct")
    .setInputCol("content")
    .setOutputCol("result")
    .setModelParam(max_new_tokens=1000)
    .setModelConfig(local_files_only=False, trust_remote_code=True)
)
result_df = phi3_transformer.transform(chat_df).collect()
display(result_df)

StatementMeta(, f9f20cd1-6e11-4d5d-af98-de2812967df7, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fc0008a8-6371-4c4d-93f3-b9b533c39f48)